In [ ]:
!pip install pandas numpy scikit-learn tensorflow openpyxl



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle


file_path = '/content/data_output (1).xlsx'
data = pd.read_excel(file_path)

# Preprocess the data
def preprocess_data(df):
    # Encode categorical features
    label_encoders = {}
    categorical_columns = ['student', 'pwd', 'isOccupation', 'isBeneficiaries']
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Normalize numerical features
    scaler = MinMaxScaler()
    df[['birthday']] = scaler.fit_transform(df[['birthday']])

    # Prepare input and output data
    X = df[['birthday', 'student', 'pwd', 'isOccupation', 'isBeneficiaries']].values
    y = to_categorical(df['programId'].values - 1)  # Subtract 1 to start IDs from 0

    return X, y, label_encoders, scaler

X, y, label_encoders, scaler = preprocess_data(data)


In [ ]:
# Calculate the correct input dimension for the Embedding layer
input_dim = max([len(le.classes_) for le in label_encoders.values()]) + 1

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=10, input_length=X.shape[1]))
model.add(SimpleRNN(units=50, activation='relu'))
model.add(Dense(units=y.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50, batch_size=10, validation_split=0.2)

# Save the model
model.save('rnn_recommendation_model.h5')

# Save the encoders and scaler
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

Epoch 1/50
80/80 [==============================] - 3s 13ms/step - loss: 2.9742 - accuracy: 0.0925 - val_loss: 2.6198 - val_accuracy: 0.0750
Epoch 2/50
80/80 [==============================] - 1s 6ms/step - loss: 2.3903 - accuracy: 0.2225 - val_loss: 2.0216 - val_accuracy: 0.3200
Epoch 3/50
80/80 [==============================] - 0s 6ms/step - loss: 1.7774 - accuracy: 0.3613 - val_loss: 1.2496 - val_accuracy: 0.4500
Epoch 4/50
80/80 [==============================] - 1s 7ms/step - loss: 1.4205 - accuracy: 0.3925 - val_loss: 1.1730 - val_accuracy: 0.4650
Epoch 5/50
80/80 [==============================] - 1s 7ms/step - loss: 1.3019 - accuracy: 0.4162 - val_loss: 1.1544 - val_accuracy: 0.4850
Epoch 6/50
80/80 [==============================] - 1s 7ms/step - loss: 1.2231 - accuracy: 0.4412 - val_loss: 1.0210 - val_accuracy: 0.5250
Epoch 7/50
80/80 [==============================] - 1s 8ms/step - loss: 1.1909 - accuracy: 0.4475 - val_loss: 1.1035 - val_accuracy: 0.5250
Epoch 8/50
80/80 [=

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
